## testing the creation of the dataset first RCM 

In [3]:
import requests
from bs4 import BeautifulSoup
import random

# Écrire les en-têtes dans le fichier s'ils n'existent pas déjà
with open('cti_rcm_generated.tsv', 'a') as file:
    if file.tell() == 0:  # Vérifier si le fichier est vide
        file.write('URL' + '\t' + 'Description' + '\t' + 'Prompt' + '\t' + 'GT' + '\n')

# Nombre de lignes à générer
nb = 0
max_records = 3  # Ajustez ici pour tester avec seulement 3 enregistrements

# Boucle pour générer les enregistrements
while nb < max_records:
    # Génération aléatoire de l'année et de l'ID pour le CVE
    year = random.choice([2023, 2024])
    sub_url = random.randint(1000, 500000)
    url = f'https://nvd.nist.gov/vuln/detail/CVE-{year}-{sub_url}/'

    try:
        # Récupérer le contenu de la page web
        response = requests.get(url)
        
        # Vérification du succès de la requête
        if response.status_code == 200:
            webpage_content = response.text
            print('Page web récupérée avec succès:', year, sub_url)

            # Analyser le contenu HTML avec BeautifulSoup
            soup = BeautifulSoup(webpage_content, 'html.parser')

            # Extraction de la description de la vulnérabilité et des CWE associés
            vuln_description = soup.find('p', {'data-testid': 'vuln-description'})
            vuln_cwes_link = soup.find('td', {'data-testid': 'vuln-CWEs-link-0'})

            # Si description et CWE sont disponibles, les traiter
            if vuln_description and vuln_cwes_link:
                description_text = vuln_description.get_text().strip()
                cwes_text = vuln_cwes_link.get_text().strip()

                # Vérifier que le texte CWE contient des données pertinentes
                if "-noinfo" not in cwes_text and 'CWE-' in cwes_text:
                    # Construction du prompt à partir de la description
                    prompt = (f"Analyze the following CVE description and map it to the appropriate CWE. "
                              f"Provide a brief justification for your choice. "
                              f"Ensure the last line of your response contains only the CWE ID. CVE Description: {description_text}")

                    # Écriture dans le fichier TSV
                    with open('cti_rcm_generated.tsv', 'a') as file:
                        file.write(f"{url}\t{description_text}\t{prompt}\t{cwes_text}\n")
                        file.flush()  # Forcer l'écriture immédiate dans le fichier
                    nb += 1
                    print(f"Enregistrement {nb} écrit : {url}")

    except Exception as e:
        print(f"Erreur avec {url}: {e}")
        continue


Page web récupérée avec succès: 2023 78955
Page web récupérée avec succès: 2024 148992
Page web récupérée avec succès: 2024 61535
Page web récupérée avec succès: 2023 171367
Page web récupérée avec succès: 2024 57463
Page web récupérée avec succès: 2024 229793
Page web récupérée avec succès: 2024 139506
Page web récupérée avec succès: 2023 353838
Page web récupérée avec succès: 2024 297861
Page web récupérée avec succès: 2024 389810
Page web récupérée avec succès: 2023 354039
Page web récupérée avec succès: 2023 119873
Page web récupérée avec succès: 2023 283453
Page web récupérée avec succès: 2024 117726
Page web récupérée avec succès: 2024 79743
Page web récupérée avec succès: 2023 343122
Page web récupérée avec succès: 2024 42119
Page web récupérée avec succès: 2024 359732
Page web récupérée avec succès: 2024 140169
Page web récupérée avec succès: 2024 234367
Page web récupérée avec succès: 2024 488140
Page web récupérée avec succès: 2023 65727
Page web récupérée avec succès: 2023 4

## testing of the generation of the dataset of vsp 

In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Charger les liens depuis un fichier TSV
try:
    links = pd.read_csv("cti_rcm_generated.tsv", sep="\t", encoding='latin1')['URL']
except FileNotFoundError:
    print("Fichier 'cti_rcm_generated.tsv' non trouvé.")
    links = []

# Initialiser le fichier de sortie avec les en-têtes si nécessaire
output_file = 'cti_vsp_generated.tsv'
with open(output_file, 'a') as file:
    if file.tell() == 0:  # Si le fichier est vide, ajouter les en-têtes
        file.write('URL\tDescription\tPrompt\tGT\n')

# Tester sur seulement deux liens
test_links = links[:2] if len(links) > 1 else links

# Parcourir les liens et collecter les données
for link in test_links:
    try:
        # Récupérer la page
        response = requests.get(link)
        
        if response.status_code == 200:
            webpage_content = response.text
            print(f"Page récupérée avec succès : {link}")
            
            # Analyser le contenu HTML avec BeautifulSoup
            soup = BeautifulSoup(webpage_content, 'html.parser')
            
            # Extraction des informations pertinentes
            vuln_description = soup.find('p', {'data-testid': 'vuln-description'})
            gt = soup.find('span', {'data-testid': 'vuln-cvss3-nist-vector'}) or \
                 soup.find('span', {'data-testid': 'vuln-cvss3-cna-vector'})
            
            # Vérification de l'existence des éléments
            if vuln_description and gt:
                description_text = vuln_description.get_text().strip()
                gt_text = gt.get_text().strip()
                
                # Génération du prompt
                prompt = (
                    "Analyze the following CVE description and calculate the CVSS v3.1 Base Score. "
                    "Determine the values for each base metric: AV, AC, PR, UI, S, C, I, and A. "
                    "Summarize each metric's value and provide the final CVSS v3.1 vector string. "
                    "Valid options for each metric are as follows: - **Attack Vector (AV)**: Network (N), Adjacent (A), "
                    "Local (L), Physical (P) - **Attack Complexity (AC)**: Low (L), High (H) - **Privileges Required (PR)**: "
                    "None (N), Low (L), High (H) - **User Interaction (UI)**: None (N), Required (R) - **Scope (S)**: "
                    "Unchanged (U), Changed (C) - **Confidentiality (C)**: None (N), Low (L), High (H) - **Integrity (I)**: "
                    "None (N), Low (L), High (H) - **Availability (A)**: None (N), Low (L), High (H) "
                    "Summarize each metric's value and provide the final CVSS v3.1 vector string. Ensure the final line of "
                    "your response contains only the CVSS v3 Vector String in the following format: "
                    "Example format: CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H "
                    f"CVE Description: {description_text}"
                )
                
                # Écriture des données dans le fichier
                with open(output_file, 'a') as file:
                    file.write(f"{link}\t{description_text}\t{prompt}\t{gt_text}\n")
                print(f"Données enregistrées pour {link}")
            else:
                print(f"Description ou GT manquant pour {link}")
        else:
            print(f"Erreur {response.status_code} pour {link}")
    except Exception as e:
        print(f"Erreur avec {link}: {e}")


Page récupérée avec succès : https://nvd.nist.gov/vuln/detail/CVE-2023-24807/
Données enregistrées pour https://nvd.nist.gov/vuln/detail/CVE-2023-24807/
Page récupérée avec succès : https://nvd.nist.gov/vuln/detail/CVE-2023-2323/
Données enregistrées pour https://nvd.nist.gov/vuln/detail/CVE-2023-2323/
